In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv(r'appian-x-iit-madras-hackathon-april-2025\train.csv')
test = pd.read_csv(r'appian-x-iit-madras-hackathon-april-2025\test.csv')

data.replace('Alone', 'Single', inplace=True)
test.replace('Absurd', 'Single', inplace=True)
data.replace('YOLO', 'Single', inplace=True)
test.replace('YOLO', 'Single', inplace=True)
test.replace('Alone', 'Single', inplace=True)
data.replace('Together', 'Married', inplace=True)
test.replace('Together', 'Married', inplace=True)

test.replace('Basic', '2n Cycle', inplace=True)
data.replace('Basic', '2n Cycle', inplace=True)
test.replace('Widow', 'Divorced', inplace=True)
data.replace('Widow', 'Divorced', inplace=True)

data['Dt_Customer_1'] = pd.to_datetime(data['Dt_Customer'],format='mixed')
data['Dt_Customer_1'] = data['Dt_Customer_1']-min(data['Dt_Customer_1'])
data['Dates']=data['Dt_Customer_1'].dt.days

test['Dt_Customer_1'] = pd.to_datetime(test['Dt_Customer'],format='mixed')
test['Dt_Customer_1'] = test['Dt_Customer_1']-min(test['Dt_Customer_1'])
test['Dates']=test['Dt_Customer_1'].dt.days
Education = {}
Marital_status = {}
A = data['Education'].unique()
B = data['Marital_Status'].unique()
# A = test['Education'].unique()
# B = test['Marital_Status'].unique()
for i, category in enumerate(A):
    l = [0]*len(A)
    l[i] = 1
    print(f"{i}: {category}")
    Education[category] = i
for i, category in enumerate(B):
    l = [0]*len(B)
    l[i] = 1
    print(f"{i}: {category}")
    Marital_status[category] = i
data['Education'] = data['Education'].map(Education)
test['Education'] = test['Education'].map(Education)
data['Marital_Status'] = data['Marital_Status'].map(Marital_status)
test['Marital_Status'] = test['Marital_Status'].map(Marital_status)

# data = pd.get_dummies(data, columns=['Marital_Status', 'Education'], drop_first=True)
# test = pd.get_dummies(test, columns=['Marital_Status', 'Education'], drop_first=True)

0: Graduation
1: PhD
2: Master
3: 2n Cycle
0: Single
1: Married
2: Divorced


In [ ]:
data.info()

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

categorical_cols = ['Education', 'Marital_Status']
numerical_cols = [
    'Year_Birth', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'Complain',
    'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
    'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases',
    'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4',
    'AcceptedCmp5', 'NumWebPurchases', 'NumCatalogPurchases',
    'NumStorePurchases', 'NumWebVisitsMonth', 'Dates'
]
lists = [
    'Year_Birth',
    'Income',
    'Kidhome',
    'Teenhome',
    'Dates',
    'Recency',
    'MntWines',
    'MntFruits',
    'MntMeatProducts',
    'MntFishProducts',
    'MntSweetProducts',
    'MntGoldProds',
    'NumWebPurchases',
    'NumCatalogPurchases',
    'NumStorePurchases',
    'NumDealsPurchases',
    'NumWebVisitsMonth',
    'AcceptedCmp1',
    'AcceptedCmp2',
    'AcceptedCmp3',
    'AcceptedCmp4',
    'AcceptedCmp5',
    'Complain',
    # 'Marital_Status_Married',
    # 'Marital_Status_Single',
    # 'Education_Graduation',
    # 'Education_Master',
    # 'Education_PhD',
    'Marital_Status',
    'Education'
    # 'Target'
]
import torch
from torch.utils.data import Dataset

class CustomerDataset(Dataset):
    def __init__(self, df, feature_cols, target_col='Target'):
        self.X = df[feature_cols].values.astype('float32')
        self.y = df[target_col].values.astype('float32')

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.X[idx], dtype=torch.float),
            'target': torch.tensor(self.y[idx], dtype=torch.float)
        }
        # return torch.tensor(self.X[idx], dtype=torch.float), torch.tensor(self.y[idx], dtype=torch.float)

means = data.mean(numeric_only=True)
default_values = means[lists[:-5]]

In [3]:
dataset = CustomerDataset(data, lists, target_col='Target')

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImputationLayer(nn.Module):
    def __init__(self, default_values):
        super(ImputationLayer, self).__init__()
        self.impute = nn.Parameter(torch.tensor(default_values, dtype=torch.float32), requires_grad=True)

    def forward(self, x):
        mask = torch.isnan(x)
        x[mask] = self.impute.expand(x.shape[0], -1)[mask]
        return x

class TrainableScaler(nn.Module):
    def __init__(self, num_features):
        super(TrainableScaler, self).__init__()
        self.mean = nn.Parameter(torch.zeros(num_features))
        self.std = nn.Parameter(torch.ones(num_features))

    def forward(self, x):
        return (x - self.mean) / (self.std + 1e-6)

class MarketResearchModel(nn.Module):
    def __init__(self, num_numeric_features, emb_sizes = [3,4], layers_list=[64, 32, 1], default_values=default_values):
        super(MarketResearchModel, self).__init__()

        # Imputation and scaling layers
        self.imputer = ImputationLayer(default_values)  # Replace with actual means
        self.scaler = TrainableScaler(num_features=num_numeric_features)

        # Embedding layers
        self.embedding_1 = nn.Embedding(num_embeddings=emb_sizes[0], embedding_dim=5)  # for class feature -6 to -3
        self.embedding_2 = nn.Embedding(num_embeddings=emb_sizes[1], embedding_dim=5)  # for class feature -3 to -1

        # Input size for FFN
        input_size = num_numeric_features + 10

        # Build feedforward layers dynamically
        layers = []
        in_dim = input_size
        for out_dim in layers_list:
            layers.append(nn.Linear(in_dim, out_dim))
            if out_dim != 1:
                layers.append(nn.BatchNorm1d(out_dim))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(0.3))
            in_dim = out_dim
        if layers_list[-1] != 1:
            layers.append(nn.Linear(in_dim, 1))
        layers.append(nn.Sigmoid())

        self.ff = nn.Sequential(*layers)

    def forward(self, x):
        x_numeric = x[:, :-2]
        # x_cat1 = x[:, -5:-3].long()
        # x_cat2 = x[:, -3:-1].long()
        x_cat1 = x[:,-2:-1].long()
        x_cat2 = x[:,-1:].long()
        # print(x_numeric.shape, x_cat1.shape, x_cat2.shape)
        x_numeric = self.imputer(x_numeric)
        x_numeric = self.scaler(x_numeric)

        emb1 = self.embedding_1(x_cat1).squeeze(1)
        emb2 = self.embedding_2(x_cat2).squeeze(1)
        # print(emb1.shape, emb2.shape, x_numeric.shape)
        x = torch.cat([x_numeric, emb1, emb2], dim=1)
        return self.ff(x).squeeze(1)


In [7]:
len(default_values)

23

In [6]:
means = data.mean(numeric_only=True)
default_values = means[lists[:-2]]
model = MarketResearchModel(num_numeric_features=len(lists)-2, emb_sizes=[3,4], layers_list=[64, 32, 1], default_values=default_values).to('cuda')

C:\Users\monar\AppData\Local\Temp\ipykernel_17244\2147856230.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.impute = nn.Parameter(torch.tensor(default_values, dtype=torch.float32), requires_grad=True)


In [ ]:
from torchinfo import summary


summary(model, input_size=(3, len(lists)), col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"])

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split

# Set random seed
torch.manual_seed(42)

# --- Training & Evaluation ---
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        x_all, y = batch["features"], batch["target"]
        x = x_all[:, :-2]
        cat1 = x_all[:, -2].long()
        cat2 = x_all[:, -1].long()
        x, y, cat1, cat2 = x.to(device), y.to(device), cat1.to(device), cat2.to(device)
        x_all = x_all.to(device)
        optimizer.zero_grad()
        preds = model(x_all)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * len(y)
    return total_loss / len(dataloader.dataset)

def evaluate(model, dataloader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            x_all, y = batch["features"], batch["target"]
            x = x_all[:, :-2]
            cat1 = x_all[:, -2].long()
            cat2 = x_all[:, -1].long()
            x, y, cat1, cat2 = x.to(device), y.to(device), cat1.to(device), cat2.to(device)
            x_all = x_all.to(device)
            pred = model(x_all)
            all_preds.append(pred.cpu())
            all_labels.append(y.cpu())

    preds = torch.cat(all_preds) > 0.5
    labels = torch.cat(all_labels)
    return accuracy_score(labels.numpy(), preds.numpy())

# --- Hyperparameter Tuning ---
def run_tuning(dataset, emb_sizes, param_grid, device='cuda' if torch.cuda.is_available() else 'cpu'):
    best_acc = 0
    best_model_state = None
    best_config = None

    val_len = int(0.2 * len(dataset))
    train_len = len(dataset) - val_len
    train_ds, val_ds = random_split(dataset, [train_len, val_len])

    for config in ParameterGrid(param_grid):
        print(f"\nTraining config: {config}")
        train_loader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=config['batch_size'])
        model = MarketResearchModel(
            num_numeric_features=dataset[0]["features"].shape[0] - 2,
            emb_sizes=emb_sizes,
            layers_list=config['layers_list']
        ).to(device)
        # print(dataset[0]["features"].shape[0] - 2, emb_sizes, config['layers_list'])
        optimizer = optim.Adam(model.parameters(), lr=config['lr'])
        criterion = nn.BCELoss()

        for epoch in range(config['epochs']):
            train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
            val_acc = evaluate(model, val_loader, device)
            print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f} | Val Acc = {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_state = model.state_dict()
            best_config = config

    print(f"\nBest Config: {best_config} | Best Validation Accuracy: {best_acc:.4f}")
    return best_model_state, best_config

# --- Example Run ---
param_grid = {
    'lr': [1e-3, 5e-4, 1e-4],                          # Learning rates
    'batch_size': [32, 64, 128],                       # Batch sizes
    'layers_list': [                                   # Network depths
        [64, 32],
        [128, 64, 32],
        [256, 128, 64, 32],
        [128, 128, 64, 32]
    ],
    'epochs': [15, 25],                                # Training duration
}

# To run: best_state, best_params = run_tuning(dataset, emb_sizes=[3, 4], param_grid=param_grid)

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Set random seed
torch.manual_seed(42)

# --- Training & Evaluation ---
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        x_all, y = batch["features"], batch["target"]
        x = x_all[:, :-2]
        cat1 = x_all[:, -2].long()
        cat2 = x_all[:, -1].long()
        x, y, cat1, cat2 = x.to(device), y.to(device), cat1.to(device), cat2.to(device)
        x_all = x_all.to(device)
        optimizer.zero_grad()
        preds = model(x_all)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * len(y)
    avg_loss = total_loss / len(dataloader.dataset)
    return avg_loss

# Reuse evaluate for both train and validation

def evaluate(model, dataloader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            x_all, y = batch["features"], batch["target"]
            x = x_all[:, :-2]
            cat1 = x_all[:, -2].long()
            cat2 = x_all[:, -1].long()
            x, y, cat1, cat2 = x.to(device), y.to(device), cat1.to(device), cat2.to(device)
            x_all = x_all.to(device)
            pred = model(x_all)
            all_preds.append(pred.cpu())
            all_labels.append(y.cpu())
    preds = torch.cat(all_preds) > 0.5
    labels = torch.cat(all_labels)
    return accuracy_score(labels.numpy(), preds.numpy())

# --- Hyperparameter Tuning ---
def run_tuning(dataset, emb_sizes, param_grid, device='cuda' if torch.cuda.is_available() else 'cpu'):
    best_global_score = 0
    best_model_state = None
    best_config = None

    val_len = int(0.2 * len(dataset))
    train_len = len(dataset) - val_len
    train_ds, val_ds = random_split(dataset, [train_len, val_len])

    for config in ParameterGrid(param_grid):
        print(f"\nTraining config: {config}")
        train_loader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=config['batch_size'])

        model = MarketResearchModel(
            num_numeric_features=dataset[0]["features"].shape[0] - 2,
            emb_sizes=emb_sizes,
            layers_list=config['layers_list']
        ).to(device)

        optimizer = optim.Adam(model.parameters(), lr=config['lr'])
        criterion = nn.BCELoss()

        best_config_score = 0
        for epoch in range(config['epochs']):
            train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
            train_acc = evaluate(model, train_loader, device)
            val_acc = evaluate(model, val_loader, device)
            epoch_score = min(train_acc, val_acc)
            best_config_score = max(best_config_score, epoch_score)
            # print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f} | Train Acc={train_acc:.4f} | Val Acc={val_acc:.4f} | Min Acc={epoch_score:.4f}")

        print(f"Best Min(train, val) accuracy for config: {best_config_score:.4f}")

        if best_config_score > best_global_score:
            best_global_score = best_config_score
            best_model_state = model.state_dict()
            best_config = config

    print(f"\nBest Config: {best_config} | Best Min Acc: {best_global_score:.4f}")
    return best_model_state, best_config

# --- Example Run ---
param_grid = {
    'lr': [1e-3, 5e-4, 1e-4],                          # Learning rates
    'batch_size': [32, 64, 128],                       # Batch sizes
    'layers_list': [                                   # Network depths
        [64, 32],
        [128, 64, 32],
        [256, 128, 64, 32],
        [128, 128, 64, 32]
    ],
    'epochs': [25, 35],                                # Training duration
}

# To run: best_state, best_params = run_tuning(dataset, emb_sizes=[3, 4], param_grid=param_grid)

In [ ]:
run_tuning(dataset, emb_sizes=[3, 4], param_grid=param_grid)


Training config: {'batch_size': 32, 'epochs': 25, 'layers_list': [64, 32], 'lr': 0.001}
Best Min(train, val) accuracy for config: 0.7987

Training config: {'batch_size': 32, 'epochs': 25, 'layers_list': [64, 32], 'lr': 0.0005}
Best Min(train, val) accuracy for config: 0.7879

Training config: {'batch_size': 32, 'epochs': 25, 'layers_list': [64, 32], 'lr': 0.0001}
Best Min(train, val) accuracy for config: 0.7321

Training config: {'batch_size': 32, 'epochs': 25, 'layers_list': [128, 64, 32], 'lr': 0.001}
Best Min(train, val) accuracy for config: 0.8051

Training config: {'batch_size': 32, 'epochs': 25, 'layers_list': [128, 64, 32], 'lr': 0.0005}
Best Min(train, val) accuracy for config: 0.7796

Training config: {'batch_size': 32, 'epochs': 25, 'layers_list': [128, 64, 32], 'lr': 0.0001}
Best Min(train, val) accuracy for config: 0.7380

Training config: {'batch_size': 32, 'epochs': 25, 'layers_list': [256, 128, 64, 32], 'lr': 0.001}
Best Min(train, val) accuracy for config: 0.8179

Trai